### 29/11/2022 - v5
This is to include LADs in Scotland as well as all the amendments made by v3 previously

In [100]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from scipy.io import mmread, mmwrite
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
# from scipy import sparse
import plotly.express as px
from scipy import sparse
import png
import math
import scipy.linalg as LA
import nodevectors
from sklearn.decomposition import PCA
import seaborn as sns
from tabulate import tabulate
from scipy.stats.mstats import winsorize

Read in MSOA node data:

In [101]:
# read in MSOA names (labels for A)
nodes = pd.read_csv("nodes_MSOA.csv")
nodes = nodes[["msoa", "index"]]

### Read in lookup table for E&W from: 

https://geoportal.statistics.gov.uk/datasets/middle-layer-super-output-area-2001-to-middle-layer-super-output-area-2011-to-local-authority-district-2011-lookup-in-england-and-wales/explore

which goes from MSOA to LAD for England and Wales. 

This is the lookup used in the just E&W stuff. The section below this deals with the Scotland data. 

In [102]:
MSOA_LAD_lookup_EW = pd.read_csv("MSOA2001_MSOA2011_LAD2011_Lookup_EW.csv")

In [103]:
# only interested in the 2011 MSOA codes (not the 2001)
MSOA_LAD_lookup_EW = MSOA_LAD_lookup_EW[["MSOA11CD", "MSOA11NM", "LAD11CD", "LAD11NM"]]

In [104]:
MSOA_LAD_lookup_EW = MSOA_LAD_lookup_EW.drop_duplicates()
MSOA_LAD_lookup_EW = MSOA_LAD_lookup_EW.dropna()

In [105]:
nodes_to_LAD_EW = nodes.merge(MSOA_LAD_lookup_EW, left_on="msoa", right_on="MSOA11CD", how = "left")

In [106]:
nodes_to_LAD_EW.shape

(8483, 6)

In [107]:
# df = nodes_to_LAD_EW[nodes_to_LAD_EW['msoa'].str.contains('E')]

#### There was one MSOA in England that the lookup table didn't have a value to match it to, so I have identified where this is and manually inputted the correct data.

In [108]:
nodes_to_LAD_EW[nodes_to_LAD_EW['msoa'].str.contains('E') & nodes_to_LAD_EW["MSOA11CD"].isna()]

,msoa,index,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM
6927,E02006928,6928,NaN,NaN,NaN,NaN


In [109]:
# manually add in the values
nodes_to_LAD_EW.iloc[6927,:] = ['E02006928', '6928', 'MSOA11CD', 'Greenwich 035', 'E09000011', 'Greenwich']

E=6791
W=410
L=1
N=1
M=1

In [110]:
nodes_to_LAD_EW = nodes_to_LAD_EW.dropna()

In [111]:
nodes_to_LAD_EW.shape
# lose the nodes that are scotland or other MSOAs outside of E&W

(7201, 6)

In [112]:
# define all the LADs in a np array so that later we can go through these values in a for loop
lads_EW = nodes_to_LAD_EW.LAD11NM.unique()

In [113]:
lads_EW.shape
# 348 LADs in E&W

(348,)

### Read in lookup table for Scotland from: 

https://www.opendata.nhs.scot/dataset/geography-codes-and-labels/resource/e3e885cc-2530-4b3c-bead-9eda9782264f

which goes from MSOA (IZ = intermediate zone) to LAD (CA = council authority) for Scotland. 

This is the lookup used in the just Scotland stuff.

In [114]:
MSOA_LAD_lookup_S = pd.read_csv("iz2011_codes_and_labels_14072022.csv")

In [115]:
MSOA_LAD_lookup_S = MSOA_LAD_lookup_S[['IntZone', 'IntZoneName', 'CA', 'CAName']]

In [116]:
MSOA_LAD_lookup_S = MSOA_LAD_lookup_S.drop_duplicates()
MSOA_LAD_lookup_S = MSOA_LAD_lookup_S.dropna()

In [117]:
MSOA_LAD_lookup_S.shape

(1279, 4)

In [118]:
nodes_to_LAD_S = nodes.merge(MSOA_LAD_lookup_S, left_on="msoa", right_on="IntZone", how = "left")
# Scotland uses IntZone as the MSOA equivalent

In [119]:
nodes_to_LAD_S = nodes_to_LAD_S.dropna()
# drops all the E&W and the 3 other MSOAs (North Ireland, Isle of Man, Channel Islands)

In [120]:
nodes_to_LAD_S.shape

(1279, 6)

In [121]:
lads_S = nodes_to_LAD_S.CAName.unique()

In [122]:
lads_S.shape

(32,)

### Combine all together
This means that all the terms in the old code (that encompassed just E&W) now is the whole UK, so we are good to go!

In [123]:
# a list of all the LADs for the UK
lads = np.array(list(lads_EW) + list(lads_S))

In [124]:
len(lads)

380

In [125]:
nodes_to_LAD_S.rename(columns = {'IntZone':'MSOA11CD', 'IntZoneName':'MSOA11NM', 'CA':'LAD11CD', 'CAName':'LAD11NM'}, inplace = True)
# renaming the Scottish ones to look like the E&W ones

In [126]:
nodes_to_LAD = nodes_to_LAD_S.append(nodes_to_LAD_EW)

In [127]:
nodes_to_LAD.shape

(8480, 6)

In [128]:
nodes_to_LAD

,msoa,index,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM
0,S02001237,1,S02001237,"Cults, Bieldside and Milltimber West",S12000033,Aberdeen City
1,S02001296,2,S02001296,"Dunecht, Durris and Drumoak",S12000034,Aberdeenshire
2,S02001236,3,S02001236,Culter,S12000033,Aberdeen City
3,S02001250,4,S02001250,City Centre East,S12000033,Aberdeen City
4,S02001261,5,S02001261,George Street,S12000033,Aberdeen City
...,...,...,...,...,...,...
8471,E02002698,8472,E02002698,East Riding of Yorkshire 015,E06000011,East Riding of Yorkshire
8472,E02005758,8473,E02005758,Hambleton 009,E07000164,Hambleton
8473,E02005757,8474,E02005757,Hambleton 008,E07000164,Hambleton
8474,E02005813,8475,E02005813,Selby 005,E07000169,Selby


In [129]:
nodes_to_LAD['index'] = nodes_to_LAD['index'].astype(int)

In [130]:
nodes_to_LAD = nodes_to_LAD.sort_values(by=['index'], ascending=True)

### Make A matrix of the concatenated 380x380 (LAD by LAD) A matrices, from row and column summing.

In [131]:
A_list = []
A_LADLAD_list = []
year_list = []

for year in range(2005,2011):
    # does 2005-2010 inclusive
    filepath = "A_for_" + str(year) + "_binary.mtx" # file name to read in 
    A = mmread(filepath) # temporary
    
    year_list.append(year)
    
    A = A.toarray()
    
    A_LADMSOA = list()
    A_LADLAD =list() # empty numpy array to fill, one row
    # to make "LAD by LAD" version of A - called A_LADLAD
    
    for lad in lads: 
        
        # find all the indexes for the given reg
        row_indexes = nodes_to_LAD.loc[nodes_to_LAD["LAD11NM"] == lad, "index"]
        # make a numpy array and subtract 1, as python indexes from 0 not 1
        row_indexes  = row_indexes.to_numpy()
        row_indexes = [int(x) - 1 for x in row_indexes]
        column_indexes = row_indexes
        
        # find all the rows with the indexes for that region, and add them together to make one row 
        row = A[row_indexes, :]
        row = np.sum(row, axis=0)
        
        # one row for each region in the matrix A_RegMSOA
        A_LADMSOA.append(row)
        
    A_LADMSOA = np.asarray(A_LADMSOA)
    
    for lad in lads: 
        
        # find all the columns with the indexes for that region, from the already row summed matrix, 
        # and add them together to make one column 
        column_indexes = nodes_to_LAD.loc[nodes_to_LAD["LAD11NM"] == lad, "index"]
        column_indexes = column_indexes.to_numpy()
        column_indexes = [int(x) - 1 for x in column_indexes]

        column = A_LADMSOA[:, column_indexes]
        column = np.sum(column, axis=1)
        
        A_LADLAD.append(column)
    
    A_LADLAD = np.asarray(A_LADLAD)
    A_LADLAD = sparse.coo_matrix(A_LADLAD)

    # add a matrix for each year to the list
    A_LADLAD_list.append(A_LADLAD)
    
#print(A_RegMSOA_list)    
A_LADLAD_all = hstack(A_LADLAD_list)

In [132]:
A_LADLAD_all.shape
# n x 6n 

(380, 2280)

In [133]:
# save all the LAD by LAD matrices

for i in range(0,6):
    mat = A_LADLAD_list[i]
    mmwrite("A_LAD_" + str(i+2005) + "_UK_v5.mtx", mat)

Read in lookup table from: 

https://www.data.gov.uk/dataset/2dc9ac85-8da2-48b4-a6f4-b4502af19c06/postcode-to-output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-may-2019-lookup-in-the-uk

has columns msoa11nm, msoa11cd, ladcd, and ladnm that I use to map from one to another. Also assuming that it is lad11nm they are using as all the other fields are 2011 dated (if they are dated). 

The next chunk of code is to take the necessary columns and rows out of to make a MSOA to LAD UK lookup. It outputs the `MSOA_LAD_UK_lookup.csv` file, which can then just be used, so the section is commented out as it doesn't need to be ran over and over.

#### Section to create the MSOA to LAD for UK lookup csv file and save it

In [134]:
bigdf = pd.read_csv("PCD_OA_LSOA_MSOA_LAD_FEB20_UK_LU.csv")

/home/uj21900/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [135]:
df1 = bigdf.iloc[:, [8,9,11,12]]

In [136]:
MSOA_LAD_UK_lookup = df1.drop_duplicates()
MSOA_LAD_UK_lookup = MSOA_LAD_UK_lookup.dropna()

In [137]:
MSOA_LAD_UK_lookup.shape

(3767, 4)

In [138]:
MSOA_LAD_UK_lookup.to_csv("MSOA_LAD_UK_lookup.csv")

#### Read in the lookup file to use it
(How to make the lookup file is the code commented out above)

In [139]:
# read in file
MSOA_LAD_UK_lookup = pd.read_csv("MSOA_LAD_UK_lookup.csv")

In [140]:
# remove the odd index column
MSOA_LAD_UK_lookup = MSOA_LAD_UK_lookup[['msoa11cd', 'ladcd', 'msoa11nm', 'ladnm']]

In [141]:
MSOA_LAD_UK_lookup.head()

,msoa11cd,ladcd,msoa11nm,ladnm
0,S02001237,S12000033,"Cults, Bieldside and Milltimber Wes",Aberdeen City
1,S02001296,S12000034,"Dunecht, Durris and Drumoak",Aberdeenshire
2,S02001236,S12000033,Culter,Aberdeen City
3,S02001250,S12000033,City Centre East,Aberdeen City
4,S02001261,S12000033,George Street,Aberdeen City


In [142]:
MSOA_LAD_UK_lookup.shape

(3767, 4)

In [143]:
nodes_to_LAD = nodes.merge(MSOA_LAD_UK_lookup, left_on="msoa", right_on="msoa11cd", how = "left")

In [144]:
nodes_to_LAD.shape

(8493, 6)

In [145]:
test = MSOA_LAD_UK_lookup["msoa11nm"]

In [146]:
test1 = test.drop_duplicates()

In [147]:
test1.shape

(3730,)

In [148]:
nodes_to_LAD = nodes_to_LAD.dropna()

In [149]:
nodes_to_LAD.shape
# lose the nodes that are scotland or other MSOAs outside of E&W

(3767, 6)

In [150]:
dataFrameOut = nodes[nodes['msoa'].str.contains('N')]
dataFrameOut.shape

(1, 2)

In [151]:
dataFrameOut

,msoa,index
1062,N99999999,1063


In [152]:
scotIZ = pd.read_csv("Intermediate_Zones_(December_2011)_Names_and_Codes_in_Scotland.csv")

In [153]:
scotIZ.head()

,FID,IZ11CD,IZ11NM
0,1,S02001236,Culter
1,2,S02001336,Peterhead Harbour
2,3,S02001237,"Cults, Bieldside and Milltimber West"
3,4,S02001238,"Cults, Bieldside and Milltimber East"
4,5,S02001337,Peterhead Ugieside


In [154]:
testing  = nodes.merge(scotIZ, right_on="IZ11CD", left_on="msoa", how="inner")

In [155]:
testing.shape

(1279, 5)

In [156]:
IZtoCA = pd.read_csv("iz2011_codes_and_labels_14072022.csv")

In [157]:
IZtoCA.head()

,IntZone,IntZoneName,CA,CAName,HSCP,HSCPName,HB,HBName,Country
0,S02001236,Culter,S12000033,Aberdeen City,S37000001,Aberdeen City,S08000020,NHS Grampian,S92000003
1,S02001237,"Cults, Bieldside and Milltimber West",S12000033,Aberdeen City,S37000001,Aberdeen City,S08000020,NHS Grampian,S92000003
2,S02001238,"Cults, Bieldside and Milltimber East",S12000033,Aberdeen City,S37000001,Aberdeen City,S08000020,NHS Grampian,S92000003
3,S02001239,Garthdee,S12000033,Aberdeen City,S37000001,Aberdeen City,S08000020,NHS Grampian,S92000003
4,S02001240,"Braeside, Mannofield, Broomhill and Seafield East",S12000033,Aberdeen City,S37000001,Aberdeen City,S08000020,NHS Grampian,S92000003


In [158]:
CA = IZtoCA[["CAName"]]
CA = CA.drop_duplicates()

In [159]:
CA.shape

(32, 1)

In [160]:
# define all the LADs in a np array so that later we can go through these values in a for loop
lads = nodes_to_LAD.LAD11NM.unique()

AttributeError: 'DataFrame' object has no attribute 'LAD11NM'

In [ ]:
lads.shape
# 348 LADs in E&W